In [1]:
# Install required libraries
!pip install pandas numpy scikit-learn plotly ipywidgets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import widgets
from IPython.display import display, HTML

# Generate synthetic telco dataset
def generate_telco_data(n_samples=1000):
    np.random.seed(42)
    data = {
        'customer_id': range(1, n_samples + 1),
        'age': np.random.randint(18, 80, n_samples),
        'tenure': np.random.randint(0, 72, n_samples),
        'monthly_charges': np.random.uniform(20, 120, n_samples),
        'total_charges': np.random.uniform(0, 8000, n_samples),
        'internet_service': np.random.choice(['DSL', 'Fiber optic', 'No'], n_samples),
        'contract': np.random.choice(['Month-to-month', 'One year', 'Two year'], n_samples),
        'payment_method': np.random.choice(['Electronic check', 'Mailed check', 'Bank transfer', 'Credit card'], n_samples),
        'monthly_usage': np.random.uniform(0, 500, n_samples)
    }

    df = pd.DataFrame(data)

    # Define churn categories based on usage and other factors
    df['churn_category'] = pd.cut(df['monthly_usage'],
                                  bins=[0, 50, 200, float('inf')],
                                  labels=['Seldom', 'Average', 'Loyal'])

    return df

# Preprocess data
def preprocess_data(df):
    # Encode categorical variables
    df_encoded = pd.get_dummies(df, columns=['internet_service', 'contract', 'payment_method'])

    # Scale numerical features
    scaler = StandardScaler()
    numerical_features = ['age', 'tenure', 'monthly_charges', 'total_charges', 'monthly_usage']
    df_encoded[numerical_features] = scaler.fit_transform(df_encoded[numerical_features])

    return df_encoded

# Train model
def train_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))

    return model

# Suggest offers based on predictions
def suggest_offer(churn_category):
    offers = {
        'Loyal': "10% discount on annual plan upgrade",
        'Average': "Free additional 5GB data for 3 months",
        'Seldom': "50% off on the next 3 months subscription"
    }
    return offers[churn_category]

# Visualize results
def plot_feature_importance(model, feature_names):
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]

    fig = go.Figure(go.Bar(
        x=importances[indices][:10],
        y=[feature_names[i] for i in indices[:10]],
        orientation='h'
    ))
    fig.update_layout(title='Top 10 Feature Importances', xaxis_title='Importance', yaxis_title='Features')
    fig.show()

# Create UI
def create_ui(model, feature_names):
    def predict_churn(age, tenure, monthly_charges, total_charges, internet_service, contract, payment_method, monthly_usage):
        input_data = pd.DataFrame({
            'age': [age],
            'tenure': [tenure],
            'monthly_charges': [monthly_charges],
            'total_charges': [total_charges],
            'monthly_usage': [monthly_usage],
            'internet_service_DSL': [1 if internet_service == 'DSL' else 0],
            'internet_service_Fiber optic': [1 if internet_service == 'Fiber optic' else 0],
            'internet_service_No': [1 if internet_service == 'No' else 0],
            'contract_Month-to-month': [1 if contract == 'Month-to-month' else 0],
            'contract_One year': [1 if contract == 'One year' else 0],
            'contract_Two year': [1 if contract == 'Two year' else 0],
            'payment_method_Bank transfer': [1 if payment_method == 'Bank transfer' else 0],
            'payment_method_Credit card': [1 if payment_method == 'Credit card' else 0],
            'payment_method_Electronic check': [1 if payment_method == 'Electronic check' else 0],
            'payment_method_Mailed check': [1 if payment_method == 'Mailed check' else 0]
        })

        prediction = model.predict(input_data)[0]
        offer = suggest_offer(prediction)

        result.value = f"Predicted category: {prediction}<br>Suggested offer: {offer}"

    age_slider = widgets.IntSlider(min=18, max=80, value=30, description='Age:')
    tenure_slider = widgets.IntSlider(min=0, max=72, value=12, description='Tenure (months):')
    monthly_charges_slider = widgets.FloatSlider(min=20, max=120, value=50, description='Monthly Charges:')
    total_charges_slider = widgets.FloatSlider(min=0, max=8000, value=1000, description='Total Charges:')
    internet_service_dropdown = widgets.Dropdown(options=['DSL', 'Fiber optic', 'No'], value='DSL', description='Internet Service:')
    contract_dropdown = widgets.Dropdown(options=['Month-to-month', 'One year', 'Two year'], value='Month-to-month', description='Contract:')
    payment_method_dropdown = widgets.Dropdown(options=['Electronic check', 'Mailed check', 'Bank transfer', 'Credit card'], value='Electronic check', description='Payment Method:')
    monthly_usage_slider = widgets.FloatSlider(min=0, max=500, value=100, description='Monthly Usage (GB):')

    predict_button = widgets.Button(description='Predict')
    result = widgets.HTML()

    predict_button.on_click(lambda _: predict_churn(
        age_slider.value,
        tenure_slider.value,
        monthly_charges_slider.value,
        total_charges_slider.value,
        internet_service_dropdown.value,
        contract_dropdown.value,
        payment_method_dropdown.value,
        monthly_usage_slider.value
    ))

    display(HTML("<h2>Customer Churn Prediction</h2>"))
    display(age_slider, tenure_slider, monthly_charges_slider, total_charges_slider, internet_service_dropdown, contract_dropdown, payment_method_dropdown, monthly_usage_slider, predict_button, result)

    plot_feature_importance(model, feature_names)

# Main execution
df = generate_telco_data()
df_encoded = preprocess_data(df)

X = df_encoded.drop(['customer_id', 'churn_category'], axis=1)
y = df_encoded['churn_category']

model = train_model(X, y)
create_ui(model, X.columns)

              precision    recall  f1-score   support

     Average       0.95      1.00      0.98        63
       Loyal       1.00      1.00      1.00       117
      Seldom       1.00      0.85      0.92        20

    accuracy                           0.98       200
   macro avg       0.98      0.95      0.97       200
weighted avg       0.99      0.98      0.98       200



IntSlider(value=30, description='Age:', max=80, min=18)

IntSlider(value=12, description='Tenure (months):', max=72)

FloatSlider(value=50.0, description='Monthly Charges:', max=120.0, min=20.0)

FloatSlider(value=1000.0, description='Total Charges:', max=8000.0)

...

Dropdown(description='Contract:', options=('Month-to-month', 'One year', 'Two year'), value='Month-to-month')

Dropdown(description='Payment Method:', options=('Electronic check', 'Mailed check', 'Bank transfer', 'Credit …

FloatSlider(value=100.0, description='Monthly Usage (GB):', max=500.0)

Button(description='Predict', style=ButtonStyle())

HTML(value='')